In [1]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import emoji
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [6]:
def remove_emojis(text):
    return emoji.replace_emoji(text, replace='')

# Loading dataset
df = pd.read_csv('/Users/arunaa/Python/Sracasam/sarcasm_tam_train.csv')


In [7]:
# Apply the remove_emojis function to the text column
df['Text'] = df['Text'].apply(remove_emojis)

# Encode the labels
label_encoder = LabelEncoder()
df['labels'] = label_encoder.fit_transform(df['labels'])

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(df['Text'], df['labels'], test_size=0.2, random_state=42)

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_val_sequences = tokenizer.texts_to_sequences(X_val)

max_length = 100
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post', truncating='post')
X_val_padded = pad_sequences(X_val_sequences, maxlen=max_length, padding='post', truncating='post')


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

# Build the CNN model
model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=max_length),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train, epochs=5, validation_data=(X_val_padded, y_val), batch_size=32)


Epoch 1/5


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


740/740 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.7516 - loss: 0.5090 - val_accuracy: 0.7908 - val_loss: 0.4410
Epoch 2/5
740/740 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.8383 - loss: 0.3584 - val_accuracy: 0.7802 - val_loss: 0.4606
Epoch 3/5
740/740 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8981 - loss: 0.2544 - val_accuracy: 0.7868 - val_loss: 0.5411
Epoch 4/5
740/740 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.9456 - loss: 0.1538 - val_accuracy: 0.7727 - val_loss: 0.6961
Epoch 5/5
740/740 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9761 - loss: 0.0773 - val_accuracy: 0.7565 - val_loss: 0.8463


In [9]:
from sklearn.metrics import accuracy_score

# Predict on the validation set
val_predictions = model.predict(X_val_padded)
val_predictions = (val_predictions > 0.5).astype(int).flatten()

# Calculate accuracy on the validation set
val_accuracy = accuracy_score(y_val, val_predictions)
print(f"Validation Accuracy: {val_accuracy:.4f}")

185/185 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Validation Accuracy: 0.7565


In [10]:
# Preprocess the test data
test_df = pd.read_csv('/Users/arunaa/Python/Sracasam/sarcasm_tam_test_without_labels.csv')
test_df['Text'] = test_df['Text'].apply(remove_emojis)
test_sequences = tokenizer.texts_to_sequences(test_df['Text'])
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')


In [11]:

# Predict on the test set
test_predictions = model.predict(test_padded)
test_predictions = (test_predictions > 0.5).astype(int).flatten()

# Save the predictions to a CSV file
test_df['predicted_labels'] = label_encoder.inverse_transform(test_predictions)
test_df.to_csv('/Users/arunaa/Python/Sracasam/predictions.csv', index=False)


199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [19]:
print("Predictions:")
for i, prediction in enumerate(test_predictions):
    print(f"Sample {i}: {prediction}")

Predictions:
Sample 0: 0
Sample 1: 0
Sample 2: 0
Sample 3: 0
Sample 4: 0
Sample 5: 1
Sample 6: 1
Sample 7: 0
Sample 8: 0
Sample 9: 1
Sample 10: 0
Sample 11: 1
Sample 12: 0
Sample 13: 1
Sample 14: 0
Sample 15: 0
Sample 16: 0
Sample 17: 1
Sample 18: 0
Sample 19: 1
Sample 20: 0
Sample 21: 1
Sample 22: 0
Sample 23: 0
Sample 24: 0
Sample 25: 1
Sample 26: 0
Sample 27: 0
Sample 28: 0
Sample 29: 0
Sample 30: 0
Sample 31: 0
Sample 32: 0
Sample 33: 1
Sample 34: 0
Sample 35: 1
Sample 36: 1
Sample 37: 0
Sample 38: 0
Sample 39: 1
Sample 40: 0
Sample 41: 0
Sample 42: 0
Sample 43: 0
Sample 44: 0
Sample 45: 0
Sample 46: 1
Sample 47: 0
Sample 48: 0
Sample 49: 1
Sample 50: 0
Sample 51: 1
Sample 52: 0
Sample 53: 0
Sample 54: 0
Sample 55: 0
Sample 56: 1
Sample 57: 0
Sample 58: 0
Sample 59: 0
Sample 60: 0
Sample 61: 0
Sample 62: 0
Sample 63: 1
Sample 64: 1
Sample 65: 0
Sample 66: 0
Sample 67: 0
Sample 68: 1
Sample 69: 1
Sample 70: 0
Sample 71: 1
Sample 72: 1
Sample 73: 1
Sample 74: 1
Sample 75: 0
Sample 76

In [16]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [17]:
from sklearn.metrics import classification_report

In [23]:
# Calculate precision, recall, F1-score, and support
report = classification_report(y_val, val_predictions, target_names=label_encoder.classes_)
print(report)

               precision    recall  f1-score   support

Non-sarcastic       0.83      0.83      0.83      4318
    Sarcastic       0.55      0.55      0.55      1596

     accuracy                           0.76      5914
    macro avg       0.69      0.69      0.69      5914
 weighted avg       0.76      0.76      0.76      5914

